### Łączenie klasyfikatorów (_ensembling_)

In [ ]:
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score, accuracy_score


from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

In [ ]:
# load data
dataset = np.loadtxt('Dane/pima-indians-diabetes.data', delimiter=",")
# split data into X and y
X = dataset[:,0:8]
Y = dataset[:,8]

print(X.shape)
print(np.mean(Y))

seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

models = [LogisticRegression(),DecisionTreeClassifier(),SVC(probability=True), LinearDiscriminantAnalysis(), QuadraticDiscriminantAnalysis(), RandomForestClassifier()]

for model in models:

    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)[:,1]
    predictions = [round(value) for value in y_pred]

    accuracy = accuracy_score(y_test, predictions)
    print("Accuracy: %.2f%%" % (accuracy * 100.0), "AUC: ", roc_auc_score(y_score=y_pred,y_true=y_test))

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
?VotingClassifier

In [ ]:
models = [LogisticRegression(),
          DecisionTreeClassifier(),
          SVC(probability=True),
          LinearDiscriminantAnalysis(), 
          QuadraticDiscriminantAnalysis()]

model = VotingClassifier(estimators=list(zip(["lr","dt","svm","lda","qda"],models)),voting="hard")
model.fit(X_train, y_train)
predictions = model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
model = VotingClassifier(estimators=list(zip(["lr","dt","svm","lda","qda"],models)),voting="soft")
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)[:,1]
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0), roc_auc_score(y_score=y_pred,y_true=y_test))

Porównanie różnych zestawów:

In [ ]:
models = [LogisticRegression(),
          SVC(probability=True),
          LinearDiscriminantAnalysis()]

for model in models:

    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)[:,1]
    predictions = [round(value) for value in y_pred]

    accuracy = accuracy_score(y_test, predictions)
    print("Accuracy: %.2f%%" % (accuracy * 100.0), "AUC: ", roc_auc_score(y_score=y_pred,y_true=y_test))

model = VotingClassifier(estimators=list(zip(["lr","svm","lda"],models)),voting="soft")
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)[:,1]
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0), roc_auc_score(y_score=y_pred,y_true=y_test))

In [ ]:
models = [LogisticRegression(),
          DecisionTreeClassifier(), 
          QuadraticDiscriminantAnalysis()]

for model in models:

    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)[:,1]
    predictions = [round(value) for value in y_pred]

    accuracy = accuracy_score(y_test, predictions)
    print("Accuracy: %.2f%%" % (accuracy * 100.0), "AUC: ", roc_auc_score(y_score=y_pred,y_true=y_test))


model = VotingClassifier(estimators=list(zip(["lr","dt","qda"],models)),voting="hard")
model.fit(X_train, y_train)
predictions = model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
?ExtraTreesClassifier

In [ ]:
model = ExtraTreesClassifier()
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)[:,1]
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0), roc_auc_score(y_score=y_pred,y_true=y_test))

# XGBoost







XGBoost jest to komitet drzew regresyjnych, który stosuje się zarówno do problemu regresji jak i klasyfikacji. Wartość zwracana przez model (dla $i$-tej obserwacji):

$$\hat{y}_i = \sum\limits_{k=1}^K f_k(x_i),$$

gdzie $f_k(\cdot)$ - wartość zwracana przez $k$-te drzewo.


Predykcja zależy od problemu:

 1) Regresja

  - predykcja: $ pred_i = \hat{y}_i $
    

2) Klasyfikacja binarna

  - predykcja: $pred_i = p(x_i) = sigmoid(\hat{y}_i) = \frac{1}{1+\exp{(-\hat{y}_i)}}$
  
3) Klasyfikacja wieloklasowa

  - predykcja: $pred_i^c =softmax(\hat{y}_i)^c = \frac{\exp{(\hat{y}_i^c)}}{\sum\limits_c\exp{(\hat{y}_i^c)}}$




Uczenie modelu - budowanie komitetu - polega na konstruowaniu kolejnych drzew w taki sposób, aby zminimalizować fukcję celu:

### $$Obj(\Theta) = L(\Theta) + \Omega(\Theta),$$

gdzie

$\Theta$ - model (zestaw konkretnych drzew)

$L(\Theta)$ - funkcja straty (miara dopasowania modelu)

$\Omega(\Theta)$ - regularyzacja (miara złozoności modelu)

Fukcja straty zależy od problemu:

 1) Regresja
    
  - funkcja straty - kwadratowa: $\sum\limits_i (y_i - \hat{y}_i)^2$


2) Klasyfikacja binarna

  - funkcja straty - log-loss (_binomial cross-entropy_): $ - \frac{1}{n}\sum\limits_{i=1}^n \big( y_i\log(p(x_i)) + (1-y_i)\log(1-p(x_i))   \big) $

3) Klasyfikacja wieloklasowa

  - funkcja straty - log-loss (_multinomial cross-entropy_): $ = -\frac{1}{n}\sum_{i=1}^n\sum\limits_c y_{ij} \log(p(x_i)_j) $


Drzewa uczone są iteracyjnie:

\begin{split}\hat{y}_i^{(0)} &= 0\\
\hat{y}_i^{(1)} &= f_1(x_i) = \hat{y}_i^{(0)} + f_1(x_i)\\
\hat{y}_i^{(2)} &= f_1(x_i) + f_2(x_i)= \hat{y}_i^{(1)} + f_2(x_i)\\
&\dots\\
\hat{y}_i^{(t)} &= \sum_{k=1}^t f_k(x_i)= \hat{y}_i^{(t-1)} + f_t(x_i)
\end{split}

W danym kroku znajdujemy drzewo, które najlepiej poprawia naszą funkcję celu.

Regularyzacja:

\begin{split}\Omega(\Theta) = \sum_{i=1}^t\Omega(f_i),  \\
\Omega(f) = \gamma T + \frac{1}{2}\lambda \sum_{j=1}^T w_j^2,
\end{split}

gdzie $T$ - liczba liści w drzewie $f$, $w_j$ wartość zwracana przez drzewo dla obseracji w $j$-tym liściu. Uwaga: wartości $w_{(\cdot)}$ są wyliczane na podstawie funkcji celu - w przybliżeniu znajdowane są takie, które dają najlepszy wynik.


W praktyce dodaje się kolejną regularyzację:

$$\hat{y}_i^{(k)} = \hat{y}_i^{(k-1)} + \eta \cdot f_k(x_i),$$

gdzie $\eta$ - _learning rate_.

https://xgboost.readthedocs.io/en/latest/model.html

In [ ]:
from xgboost.sklearn import XGBClassifier

In [ ]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

y_pred = model.predict_proba(X_test)[:,1]

predictions = np.round(y_pred)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0), roc_auc_score(y_score=y_pred,y_true=y_test))

In [ ]:
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)[:,1]

predictions = np.round(y_pred)

accuracy = accuracy_score(y_test, predictions)

print("Accuracy: %.2f%%" % (accuracy * 100.0))
print(roc_auc_score(y_score=y_pred,y_true=y_test))

In [ ]:
model.get_params()

## Jak szukać parametrów?


Co do zasady: zaczynamy od małej liczby drzew i dużego learning rate. Natomiast nic nie stoi na przeszkodzie, żeby sobie znaleźć sensowne te wartości:

In [ ]:
xgb = XGBClassifier()
params = {
    "n_estimators": [3,5,10,20,50],
    "learning_rate": [0.1, 0.2, 0.3]
}

gscv1 = GridSearchCV(param_grid=params, estimator=xgb,cv=20)
gscv1.fit(X_train,y_train)
s = gscv1.grid_scores_
s.sort(key= lambda x: x[1],reverse=True)
s

Zatem widzimy, że 5 drzew daje sensowne rezultaty, a jednocześnie to bardzo mała liczba -> podczas optymalizowania kolejnych parametrów model będzie szybko się uczył. Natomiast trzeba zwrócić uwagę, że różnice nie są duże, więc wzięcie 20 drzew też miałoby sens, bo jednak troche wiecej drzew daje lepszą stabilonść modelu, a 20 to nie jest jakoś bardzo dużo

Szukamy kolejnych parametrów.

In [ ]:
# inicjujemy przy uzyciu poprzednich najlepszych parametrow i badamy kolejne parametry
xgb = XGBClassifier(**gscv1.best_params_) #LUB gscv.best_estimator_.get_params()

params = {"max_depth": [3,4,5],
         "min_child_weight": [3, 5, 7, 10 ,15, 20]}
gscv2 = GridSearchCV(param_grid=params, estimator=xgb,cv=20)
gscv2.fit(X_train,y_train)
s = gscv2.grid_scores_
s.sort(key= lambda x: x[1],reverse=True)
s

Skrajne wartości z naszego zakresu wyszły najlepiej - trzeba poszerzyć siatkę:

In [ ]:
xgb = XGBClassifier(**gscv1.best_params_)
params = {"max_depth": [2,3,4,5,6,7,8],
         "min_child_weight": [3, 5, 7, 10 ,15, 20,25,30,40,50]}
gscv2 = GridSearchCV(param_grid=params, estimator=xgb,cv=20)
gscv2.fit(X_train,y_train)
s = gscv2.grid_scores_
s.sort(key= lambda x: x[1],reverse=True)
s

In [ ]:
# TU wszystko jedna skad wezmiemy parametry - albo gscv2.best_estimator_.get_params() albo gscv2.best_params,
# bo i tak potrzebujemy wziac jako stale te z drugiego kroku optymalizacji a z pierwszego optymalizujemy
# ale gdybysmy chcieli miec stale optymalne z krokow 1 i 2 to trzeba wziac gscv2.best_estimator_.get_params(),
# bo inaczej wzielibysmy optymalne z kroku 2 a domyslne z 1
xgb = XGBClassifier(**gscv2.best_estimator_.get_params())
params = {
    "n_estimators": [25,50,100,300,500,1000],
    "learning_rate": [0.01, 0.05, 0.1, 0.15, 0.2]
}

gscv3 = GridSearchCV(param_grid=params, estimator=xgb,cv=20)
gscv3.fit(X_train,y_train) 

s = gscv3.grid_scores_
s.sort(key= lambda x: x[1],reverse=True)
s

In [ ]:
prob = gscv3.best_estimator_.predict_proba(X_test)[:,1]

print(accuracy_score(y_pred=prob>0.5,y_true=y_test), roc_auc_score(y_score=prob,y_true=y_test))

In [ ]:
gscv3.best_estimator_.get_params()

### Czyli po optymalizacji wynik jest gorszy...

### Dlaczego tak sie stało?

Rzućmy okiem na aspekt statystyczny.

In [ ]:
X_test.shape

In [ ]:
for seed in range(10):

    test_size = 0.33
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
    
    model = XGBClassifier()
    model.fit(X_train, y_train)
    
    y_pred = model.predict_proba(X_test)[:,1]
    
    predictions = [round(value) for value in y_pred]
    
    accuracy = accuracy_score(y_test, predictions)
    print("Accuracy: %.2f%%" % (accuracy * 100.0), roc_auc_score(y_score=y_pred,y_true=y_test))

Widzimy, że w zależności od podziału accuracy waha w przedziale (73%,78%) -> TEN ZBIÓR JEST ZA MAŁY ŻEBY RZETELNIE PORÓWNYWAĆ XGBOOSTA DLA RÓŻNYCH PARAMETRÓW. Żeby móc stwierdzić, że jakieś parametry są istotnie lepsze, musiały one dać accuracy (na oko) 3 % większe niż średnia.

## Cace study: dane "adults".

In [ ]:
# Wczytaj dane treningowe i testowe

import pandas as pd
train_set = pd.read_csv('Dane/adult.data', header = None)
test_set = pd.read_csv('Dane/adult.test',skiprows = 1, header = None) # Make sure to skip a row for the test set

In [ ]:
train_set.head(20)

In [ ]:
col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 
              'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
             'wage_class']
train_set.columns = col_labels
test_set.columns = col_labels

In [ ]:
#Wyświetl informacje o typach zmiennych:

train_set.info()

In [ ]:
# Usun braki danych (oznaczone jako ' ?')

train = train_set.replace(' ?', np.nan).dropna()
test = test_set.replace(' ?', np.nan).dropna()

train.shape

In [ ]:
# Polacz zbiory

dataset = pd.concat([train,test])
print(dataset.shape)

In [ ]:
# Zamien zmienna objasniana na binarna
dataset['wage_class'] = dataset.wage_class.replace({' <=50K.': 0,' <=50K':0, ' >50K.':1, ' >50K':1})

In [ ]:
#Usun zmienna fnlwgt
dataset.drop(["fnlwgt"],axis=1,inplace=True)

In [ ]:
pd.crosstab(dataset.education,dataset.education_num)

In [ ]:
#Zaproponuj sposbob obluzenia informacji dotyczących edukacji
dataset.drop(["education"],axis=1,inplace=True)

In [ ]:
dataset.head()

In [ ]:
#Wypisz informacje o rozkladach wszystkich zmiennych (nie tylko numerycznych)
dataset.describe(include="all")

In [ ]:
# Wypisz czestosci poszczegolnych narodowosci

dataset.native_country.value_counts()

In [ ]:
# Przeanalizuj rozklad y w zaleznosci od narodowosci:
# 1) rysujac histogramy w podgrupach wyznaczonych przez narodowosc

import matplotlib.pyplot as plt
%matplotlib inline

dataset['wage_class'].hist(by=dataset['native_country'],figsize=(12,12))
plt.show()

In [ ]:
# 2) rysujac histogram proporcji w grupach
x = dataset.groupby('native_country')["wage_class"].mean()
x.hist()

In [ ]:
# Zaproponuj sposob obsluzenia tej zmiennej

d = dict(pd.cut(x[x.index!=" United-States"],5,labels=range(5)))
d

In [ ]:
dataset['native_country'] = dataset['native_country'].replace(d)

In [ ]:
dataset.head()

In [ ]:
# Zamien zmienne nominalne na numeryczne 
#(tak zeby zmienna o k-wartosciach byla reprezentowana przez k-1 zmiennych binarnych)
dataset = pd.get_dummies(dataset,drop_first=True)
dataset.head()

In [ ]:
train.shape

In [ ]:
train = dataset.iloc[:train.shape[0]]
test = dataset.iloc[train.shape[0]:]

X_train = train.drop("wage_class",axis=1)
y_train = train.wage_class

X_test = test.drop("wage_class",axis=1)
y_test = test.wage_class

X_train.shape

In [ ]:
np.mean(y_train)

### Zaczynamy modelowanie

Zaczynamy od określenia punktu wyjścia. Naucz i przetestuj klasyfikator XGBoost. Interesują nas: accuracy orac AUC.

Dla pełniejszego obrazu jeszcze wcześniej przeanalizujmy też jakość drzewa decyzyjnego:

Domyślne:

In [ ]:
m = DecisionTreeClassifier()
m.fit(X_train,y_train)

accuracy_score(y_test,m.predict(X_test)), roc_auc_score(y_test,m.predict_proba(X_test)[:,1])

Zoptymalizowane:

In [ ]:
from sklearn.model_selection import GridSearchCV

dt = DecisionTreeClassifier()
params = {
    "criterion": ["gini","entropy"],
    "max_depth": [3,4,5,6,7,8,9,10],
    "min_samples_leaf": [5,10,15,20,30,50]
}

gscv = GridSearchCV(param_grid=params, estimator=dt,cv=20)
gscv.fit(X_train,y_train)
s = gscv.grid_scores_
s.sort(key=lambda x: x[1], reverse= True)
s

In [ ]:
m = gscv.best_estimator_
m.fit(X_train,y_train)

accuracy_score(y_test,m.predict(X_test)), roc_auc_score(y_test,m.predict_proba(X_test)[:,1])

In [ ]:
xgb = XGBClassifier()
xgb.fit(X_train,y_train)

accuracy_score(y_test,xgb.predict(X_test)), roc_auc_score(y_test,xgb.predict_proba(X_test)[:,1])

## Zadanie. Zbadaj schematy optymalizacji parametrów.

Podejście 1

Ustalamy małą liczbę drzew i duży learning rate. Następnie iteracyjnie optymalizujemy parametry: maksymalana głębokość drzew, minimalna liczb (waga) dzieci, parametr regularyzacji lambda, parametr gamma. Nastepnie zoptymalizuj liczbę drzew, learning rate i jeszcze raz pozostałe parametry. Przetestuj najlepszy model na zbiorze testowym.

Podejście 2

Ustalamy małą liczbę drzew i duży learning rate. Następnie "grid searchem" zoptymalizuj pozostałe parametry. Powtórz oba kroki dwukrotnie, uwzględniając wyniki z pierwszego kroku - zmień siatki parametrów tak, aby ich zakres nie był za duży i pokrywał okolice najlepszych parametrów.

Podejści 3

Ustalamy małą liczbę drzew i duży learning rate. Następnie randomizowany grid search po pozostałych parametrach dwuetapowy: w pierwszym etapie po prostu randomizowany grid search, a w drugim etapie zwykły grid search z mała siatką pokrywającą okolice najlepszych wyników. Na koniec zwiekszamy liczbę drzew zmiejszamy learning rate.


Inne sensowne podejście (Podobne do 1 i 2, lecz skrócone):
https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/

Podejscie 1

In [ ]:
xbg = XGBClassifier()
params = {"max_depth":[5,7,10,12,15,20]}
cv = GridSearchCV(cv=5, estimator=xbg,param_grid=params)
cv.fit(X_train,y_train)
cv.grid_scores_

In [ ]:
xbg = XGBClassifier().set_params(**cv.best_estimator_.get_params())
params = {"min_child_weight":[5,10,15,20,30,50]}
cv = GridSearchCV(cv=5, estimator=xbg,param_grid=params)
cv.fit(X_train,y_train)
cv.grid_scores_

In [ ]:
xbg = XGBClassifier().set_params(**cv.best_estimator_.get_params())
params = {"reg_lambda":[0.5,1,2,3]}
cv = GridSearchCV(cv=5, estimator=xbg,param_grid=params)
cv.fit(X_train,y_train)
cv.grid_scores_

In [ ]:
xbg = XGBClassifier().set_params(**cv.best_estimator_.get_params())
params = {"n_estimators":[20,50,100,200,300]}
cv = GridSearchCV(cv=5, estimator=xbg,param_grid=params)
cv.fit(X_train,y_train)
cv.grid_scores_

In [ ]:
xbg = XGBClassifier().set_params(**cv.best_estimator_.get_params())
params = {"learning_rate":[0.01,0.05,0.1,0.15,0.2]}
cv = GridSearchCV(cv=5, estimator=xbg,param_grid=params)
cv.fit(X_train,y_train)
cv.grid_scores_

In [ ]:
print(cv.best_params_)
accuracy_score(y_test,cv.best_estimator_.predict(X_test)), roc_auc_score(y_test,cv.best_estimator_.predict_proba(X_test)[:,1])

Podejscie 2

In [ ]:
xbg = XGBClassifier(learning_rate=0.2,n_estimators=20)
params = {"max_depth":[5,7,10,12,15,20],
         "min_child_weight":[5,10,15,20,30,50],
         "reg_lambda":[0.5,1,2,3]}
cv = GridSearchCV(cv=5, estimator=xbg,param_grid=params)
cv.fit(X_train,y_train,v)
cv.grid_scores_
cv.grid_scores_.sort(key= lambda x: x[1],reverse=True)
cv.grid_scores_

In [ ]:
xbg = XGBClassifier().set_params(**cv.best_estimator_.get_params())
params = {"learning_rate":[0.01,0.05,0.1,0.15,0.2],
         "n_estimators":[30,50,100,200]}
cv = GridSearchCV(cv=5, estimator=xbg,param_grid=params)
cv.fit(X_train,y_train)
cv.grid_scores_.sort(key= lambda x: x[1],reverse=True)
cv.grid_scores_

In [ ]:
print(cv.best_params_)
accuracy_score(y_test,cv.best_estimator_.predict(X_test)), roc_auc_score(y_test,cv.best_estimator_.predict_proba(X_test)[:,1])

Podejscie 3

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as ss

In [ ]:
xgb = XGBClassifier()
params = {"n_estimators":ss.randint(10,300),
         "learning_rate":ss.uniform(0.01,0.3),
          "max_depth": ss.randint(5,30),
         "min_child_weight":ss.randint(5,50),
         "reg_lambda":ss.uniform(0.1,3)}

cv = RandomizedSearchCV(cv=5, estimator=xbg,param_distributions=params,n_iter=30,verbose=100)
cv.fit(X_train,y_train)
cv.grid_scores_.sort(key= lambda x: x[1],reverse=True)
cv.grid_scores_

In [ ]:
x = cv.grid_scores_.copy()   ### TRZEBA SKOPIOWAC BO INACZEJ NIE DZIALA
x.sort(key= lambda x: x[1],reverse=True)
x

In [ ]:
print(cv.best_params_)
accuracy_score(y_test,cv.best_estimator_.predict(X_test)), roc_auc_score(y_test,cv.best_estimator_.predict_proba(X_test)[:,1])

# Braki danych

### Jak obsługiwać braki danych?

### Propozycja (w przypadku zmiennej numerycznej):

1. Gdy dla danej zmiennej braków jest bardzo mało (np. 2% zbioru), wyrzucić obserwacje zawieracjące te braki.

2. Gdy dla danej zmiennej braków jest bardzo dużo (ponad 60%), usunąć z danych tę zmienną

3. W pozostałych przypadkach uzupełnić braki średnią.


<br>

<br>

<br>

<br>

<br>

<br>

### Nie, nie, nie i jeszcze raz nie.

# Podejście rzetelne:

1) Spróbować wyjaśnić pochodzenie braków
  - może wynikają z błędu w jakimś systemmie -> trzeba naprawić system
  - może ktoś nie wiedzał jakie wartości wprowadzić -> trzeba stworzyć lepszą (jakąkolwiek) dokumentację/instrukcję
  - itd.
  - jak to wygląda w praktyce? ...
  - wyjaśnienie jest super, ale z reguły nierealizowalne ( co nie znaczy, że nie warto próbować), dlatego trzeba sobie radzić
  
2) Spróbować wyjaśnić znaczenie braku
  - brak w kolumnie "liczba dzieci" -> może oznaczać wartość 0
  - brak w kolumnie "wykształcenie" -> może oznaczać, że osoba nie ma nawet podstawowego
  - brak może wynikać z wartości innej zmiennej -> czy posiadasz dzieci - nie, liczba dzieci - brak
  - wówczas braki możemy uzupełnić 
  
3) Jeżeli rozważamy zmienną kategoryczną:
  - warto rozważyć nadanie brakom klasy "brak" i traktować ją jak normalna wartość
  - można tez upełnić wartością występującą najczęściej lub wartością losową
  - jeżeli braki stanowią przytłaczającą większość -> można zamienić zmienną na binarną o wartości 1, gdy wystepuje brak lub 0 wpp.

4) Jeżeli rozważamy zmienną numeryczną:
  - można uzupełnić średnią -> gdy zmienna ma rozkład symetryczny
  - można uzupełnić medianą lub modą -> gdy zmienna ma rozkład skośny  
  - można tez upełnić wartością występującą wartością losową (z rozsądnego rozkładu)
  - jeżeli braki stanowią przytłaczającą większość -> można zamienić zmienną na binarną o wartości 1, gdy wystepuje brak lub 0 wpp.
  - można skategoryzować zmienną - wartości numeryczne pogrupować na przedziały i dodać kategorię "brak"
  - uzupełnić braki i jednocześnie dodać do danych zmienną binarną o wartości 1, gdy wystepuje brak lub 0 wpp.
  
5) Usunąć obserwacje z brakiem jeśli jest ich mało

6) Usunąć zmienną jeśli braków jest dużo

7) Uzupełnić używając modelu predykcyjnego z wykorzystaniem pozostałych zmiennych


# Case study 


Opis problemu i danych:
https://discuss.analyticsvidhya.com/t/hackathon-3-x-predict-customer-worth-for-happy-customer-bank/3802

Interesują nac dwie miary jakości: 
- AUC
- Zysk, liczony w następujący sposób:

  - przypisanie obserwacji kosztuje nas 100 "zł"
  - Trafienie predykcą w klasę 1 przynosi nam 1000 "zł" zarobku.

Cel: osiągnąć jak największy zysk.

In [ ]:
import pandas as pd
data = pd.read_csv('Dane/Dataset/Train_nyOWmfK.csv',encoding="latin1")
print(train.shape)

**Przykładowa** transformacja danych - nie musi być wcale najlepsza

In [ ]:
from sklearn.preprocessing import LabelEncoder

def transform_data(x,k_most_common_cities=10, k_most_common_Employer_Name=10,
                    k_most_common_Salary_Account = 10):
    
    y = x.drop("ID",axis=1,inplace=False)
    
    y["Age"] = [115-int(s[-2:]) for s in x.DOB]
    y.drop("DOB",axis=1,inplace=True)
 

    y.drop(["City","Employer_Name"],axis=1,inplace=True)    
    


    #most_common_cities = x.City.value_counts()[:k_most_common_cities]
    #y.City = x.City.apply(lambda c: c if c in most_common_cities else "Other")
 

    y.drop("Lead_Creation_Date",axis=1,inplace=True)
    #d_max = max([datetime.datetime.strptime(d, "%d-%b-%y") for d in x.Lead_Creation_Date])
    #def Lead_Creation_time_distance(d):
    #    d = datetime.datetime.strptime(d, "%d-%b-%y")
    #    dif = d_max-d
    #    return dif.days
    #y.Lead_Creation_Date = x.Lead_Creation_Date.apply(Lead_Creation_time_distance)
    
    y.Loan_Amount_Applied.fillna(x['Loan_Amount_Applied'].median(), inplace=True) 
    
    #most_common_Employer_Name = x.Employer_Name.value_counts()[:k_most_common_Employer_Name]
    #y.Employer_Name = x.Employer_Name.apply(lambda c: c if c in most_common_Employer_Name else "Other")
  
    y.Existing_EMI.fillna(0, inplace=True)

    y.drop("Salary_Account",axis=1,inplace=True)
    #most_common_Salary_Account = x.Salary_Account.value_counts()[:k_most_common_Salary_Account]
    #y.Salary_Account = x.Salary_Account.apply(lambda c: c if c in most_common_Salary_Account else "Other")
  
    #y.Loan_Amount_Submitted.fillna(0) # juz bez sprawdzania - warto stworzyc dodatkowa zmienna 0-1
    
    y.Processing_Fee = x.Processing_Fee.apply(lambda r: 1 if pd.isnull(r) else 0)
    y.Interest_Rate = x.Interest_Rate.apply(lambda r: 1 if pd.isnull(r) else 0)
    
    y.Loan_Tenure_Applied = x.Loan_Tenure_Applied.fillna(x.Loan_Tenure_Applied.median())
    
    
    
    y.Loan_Amount_Submitted = x.Loan_Amount_Submitted.apply(lambda r: 1 if pd.isnull(r) else 0)
    
    y.Loan_Tenure_Submitted = x.Loan_Tenure_Submitted.apply(lambda r: 1 if pd.isnull(r) else 0)
    
    y.EMI_Loan_Submitted = x.EMI_Loan_Submitted.apply(lambda r: 1 if pd.isnull(r) else 0)
 
    #most_common_Source = x.Source.value_counts()[:2]
    #y.Source = x.Source.apply(lambda c: c if c in most_common_Source else "Other")
    

    y['Source'] = x['Source'].apply(lambda r: 'others' if r not in ['S122','S133'] else r)
  
    le = LabelEncoder()
    var_to_encode = ['Device_Type','Filled_Form','Gender','Var1','Var2','Mobile_Verified','Source']
    for col in var_to_encode:
        y[col] = le.fit_transform(y[col])

        
    y = pd.get_dummies(y, columns=var_to_encode)
    
    
    return y

In [ ]:
data = transform_data(data)

test_size = 20000
X_train, X_test, y_train, y_test = train_test_split(data.drop(["LoggedIn","Disbursed"],axis=1,inplace=False),data.Disbursed,test_size=test_size,random_state=123)

In [ ]:
X_train.shape

In [ ]:
np.mean(y_train)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate_pipeline(clf,treshold):
    """
    Funkcja wypisuje rozne miary na podstawie predykcji
    """
    clf.fit(X_train,y_train)
    pred = clf.predict_proba(X_test)[:,1]
    
    print(1000*np.sum(((pred>treshold)==y_test) & (y_test==1))-100*np.sum(pred>treshold))
    print(np.mean((pred>treshold)==y_test))
    print(roc_auc_score(y_score=pred,y_true=y_test))
    print(precision_score(y_pred=pred>treshold,y_true=y_test), recall_score(y_pred=pred>treshold,y_true=y_test), f1_score(y_pred=pred>treshold,y_true=y_test))

Przykladowe próby rozwiązania. Nie ma tu jakiegoś rozwiązania "najlpeszego", ale ogólnie XGBoost przoduje.

In [ ]:
pca = PCA()
logreg = LogisticRegression()
sc = StandardScaler()

log_reg = Pipeline([("SS",sc),("PCA",pca),("LogReg",logreg)])

log_reg.set_params(PCA__n_components=39)
log_reg.fit(X_train,y_train)

clf = log_reg

evaluate_pipeline(clf,0.06)

In [ ]:
evaluate_pipeline(clf,0.04)

In [ ]:
evaluate_pipeline(clf,0.1)

In [ ]:
from xgboost.sklearn import XGBClassifier
xgb = XGBClassifier()

evaluate_pipeline(xgb,0.1)

In [ ]:
params = {
    "n_estimators": [50,100,500],
    "max_depth": [5,10]
}
gscv = GridSearchCV(xgb,params,"roc_auc")#,n)
gscv.fit(X_train,y_train)
gscv.grid_scores_

In [ ]:
xgb = gscv.best_estimator_

clf = xgb

evaluate_pipeline(clf,0.08)

In [ ]:
xgb = gscv.best_estimator_

clf = xgb

evaluate_pipeline(clf,0.0845)

In [ ]:
clf = gscv.best_estimator_

zyski = []
for treshold in np.linspace(0.05,0.1,num=20):
    clf.fit(X_train,y_train)
    pred = clf.predict_proba(X_test)[:,1]
    zyski.append(1000*np.sum(((pred>treshold)==y_test) & (y_test==1))-100*np.sum(pred>treshold))

In [ ]:
plt.plot(zyski)
plt.show()

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis

In [ ]:
evaluate_pipeline(LinearDiscriminantAnalysis(),0.05)